# Retrieval
- 검색은 챗봇이 훈련 데이터 외부의 데이터로 응답을 보강하는 일반적인 기술입니다. 
- 이 섹션에서는 챗봇의 맥락에서 검색을 구현하는 방법을 다룰 것입니다. 
- 하지만 검색은 매우 미묘하고 깊은 주제임을 알아두는 것이 좋습니다 
- 문서의 다른 부분을 탐색하여 더 깊이 있는 내용을 살펴보도록 권장합니다!

In [21]:
from dotenv import load_dotenv
load_dotenv('../dot.env')
import os
import getpass


def _set_if_undefined(var: str):
    # 주어진 환경 변수가 설정되어 있지 않다면 사용자에게 입력을 요청하여 설정합니다.
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


# OPENAI_API_KEY 환경 변수가 설정되어 있지 않으면 사용자에게 입력을 요청합니다.
_set_if_undefined("OPENAI_API_KEY")
# LANGCHAIN_API_KEY 환경 변수가 설정되어 있지 않으면 사용자에게 입력을 요청합니다.
_set_if_undefined("LANGCHAIN_API_KEY")
# TAVILY_API_KEY 환경 변수가 설정되어 있지 않으면 사용자에게 입력을 요청합니다.
_set_if_undefined("TAVILY_API_KEY")

# LangSmith 추적 기능을 활성화합니다. (선택적)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Lanchain_docs_Chatbots_retrieval"



In [3]:
# %pip install --upgrade --quiet langchain langchain-openai langchain-chroma beautifulsoup4

In [4]:
from langchain_openai import ChatOpenAI

# chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)
chat = ChatOpenAI(base_url="http://localhost:1234/v1",
                  api_key="lm-studio",
                  model =  "asiansoul_q8_0/Joah-Remix-Llama-3-KoEn-8B-Reborn-8B-Q8_0",
                  temperature=0.0)

아래의 예제는 langsmith 공식문서의 내용을 바탕으로 답변하는 검색기 예제입니다.

In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
data = loader.load()

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

# k is the number of chunks to retrieve
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("Can LangSmith help test my LLM applications?")

docs

[Document(page_content='Skip to main contentGo to API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'}),
 Document(page_content='Get started with LangSmith | \uf8

In [6]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context. 
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know" and please answer in Korean:

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

아래의 prompt를 확인하면 document_chain에 들어가는 변수들 간의 관계를 알 수 있습니다.

특히 중요한 변수는 context와 messages입니다. 
context는 위에서 정의한 vectorstore에서 가져온 문서들을 말하며, 
messages는 사용자의 질문에 대한 답변을 만들어내는 변수입니다. 
즉, 사용자가 질문한 내용을 바탕으로 문서를 검색하고, 그 결과를 바탕으로 답변을 만들어내는 과정을 거쳐야 합니다.

In [7]:
question_answering_prompt

ChatPromptTemplate(input_variables=['context', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the user\'s questions based on the below context. \nIf the context doesn\'t contain any relevant information to the question, don\'t make something up and just say "I don\'t know" and please answer in Korean:\n\n<context>\n{context}\n</context>\n')), MessagesPlaceholder(variable_name='messages')])

In [8]:
from langchain_core.messages import HumanMessage

document_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content="Can LangSmith help test my LLM applications?")
        ],
    }
)

'네, LangSmith는 LLM 애플리케이션을 테스트하는 데 도움이 될 수 있습니다. LangSmith를 사용하면 LLM 애플리케이션의 성능과 정확도를 평가할 수 있으며, 이를 통해 신속하고 자신감 있게 배포할 수 있습니다. 또한 LangSmith는 다양한 방법으로 트레이스를 로그할 수 있도록 제공하여 개발자들이 문제를 더 쉽게 찾고 해결할 수 있도록 도와줍니다. LangSmith의 사용은 LLM 애플리케이션을 테스트하고 개선하는 데 매우 유용합니다.'

아래는 langsmith docs를 context로 주지 않았을 때의 답변입니다.
- 관련 없는 내용이 반환

In [9]:
document_chain.invoke(
    {
        "context": [],
        "messages": [
            HumanMessage(content="Can LangSmith help test my LLM applications?")
        ],
    }
)

'LangSmith는 AI 기반의 언어 모델로, LLM(Lawyer Licensing Exam)과 같은 법률 전문 시험을 직접적으로 도와드릴 수 있는 기능은 없습니다. 그러나 LangSmith를 사용하여 영어 문법, 논리적 사고, 문제해결 능력 등 LLM에서 요구되는 기본적인 언어 및 인지 능력을 향상시킬 수는 있습니다. 또한, LangSmith를 활용하여 LLM과 관련된 법률 용어와 개념을 학습하고 이해하는 데 도움이 될 수 있습니다. 하지만, LLM의 실제 문제해결 능력과 전문 지식을 평가하는 것은 LangSmith가 아닌 사람의 역할입니다. 따라서, LangSmith는 LLM 준비에 도움을 줄 수 있지만, 직접적인 시험 지원은 불가능합니다.'

이제 위에서 생성한 doc chain과 검색기를 연결해보겠습니다.

In [10]:
# from typing import Dict

# from langchain_core.runnables import RunnablePassthrough

# # 'params' 사전에서 마지막 메시지의 내용을 추출하는 함수
# def parse_retriever_input(params: Dict):
#     return params["messages"][-1].content

# # 검색 체인을 설정합니다. 이 체인은 입력된 메시지를 처리하여 적절한 문서를 검색하고,
# # 그 결과를 document_chain에 전달하여 최종 답변을 생성합니다.
# ''' RunnablePassthrough.assign은 체인의 다음 단계로 데이터를 전달하는 데 사용됩니다.
#  document_chain의 앞단에 parse_retriever_input과 retriever를 연결하여 
#  마지막 메시지를 검색기에 전달하고, 그 결과를 document_chain에 전달하여 답변을 생성합니다.'''

# retrieval_chain = RunnablePassthrough.assign(
#     context=parse_retriever_input | retriever,  # 마지막 메시지를 검색기에 전달
# ).assign(
#     answer=document_chain,  # 검색 결과를 document_chain에 전달하여 답변 생성
# )

from typing import Dict

from langchain_core.runnables import RunnablePassthrough


def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,
).assign(
    answer=document_chain,
)

In [11]:
retrieval_chain.__dict__

{'name': None,
 'first': RunnableAssign(mapper={
   context: RunnableLambda(parse_retriever_input)
            | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x168a74a10>)
 }),
 'middle': [],
 'last': RunnableAssign(mapper={
   answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
             context: RunnableLambda(format_docs)
           }), config={'run_name': 'format_inputs'})
           | ChatPromptTemplate(input_variables=['context', 'messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the user\'s questions ba

In [12]:
retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Can LangSmith help test my LLM applications?")
        ],
    }
)

{'messages': [HumanMessage(content='Can LangSmith help test my LLM applications?')],
 'context': [Document(page_content='Skip to main contentGo to API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith |

# 쿼리 변환
- 우리의 검색 체인은 LangSmith에 대한 질문에 답할 수 있지만, 문제는 챗봇이 사용자와 대화식으로 상호작용하기 때문에 후속 질문을 처리해야 한다는 점입니다. 
- 현재 형태의 체인은 이 문제를 해결하기 어렵습니다. 
- 예를 들어, 원래 질문에 대한 후속 질문으로 "더 알려주세요!"와 같은 질문을 고려해보세요. 검색기에 해당 쿼리를 직접 전달하면 LLM 애플리케이션 테스트와 관련 없는 문서가 반환됩니다:

In [13]:
retriever.invoke("Tell me more!")

[Document(page_content='Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'}),
 Document(page_content='}],        model: "gpt-3.5-turbo",    });    return result.choices[0].message.content;});await pipeline("Hello, world!")// Out: Hello there! How can I assist you today?View a sample output trace.Learn more about tracing in the how-to guides.5. Run your first evaluation‚ÄãEvaluation requires a system to test, data to serve as test cases, and optionally evaluators to grade the results. Here we use a built-in accuracy evaluator.PythonTypeScriptfrom langsmith im

In [14]:
from langchain_core.messages import AIMessage, HumanMessage

query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
        ),
    ]
)
#위 query_transform_prompt는 message 내의 내서를 바탕으로 쿼리를 만들어내는 체인입니다. 

query_transformation_chain = query_transform_prompt | chat

query_transformation_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Can LangSmith help test my LLM applications?"),
            AIMessage(
                content="Yes, LangSmith can help test and evaluate your LLM applications. It allows you to quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs. Additionally, LangSmith can be used to monitor your application, log all traces, visualize latency and token usage statistics, and troubleshoot specific issues as they arise."
            ),
            HumanMessage(content="Tell me more!"),
        ],
    }
)
#query_transformation_chain은 주어진 messages를 바탕으로 "LangSmith LLM application testing and evaluation"라는 검색기를 위한 쿼리를 만들었습니다.

AIMessage(content='"LangSmith LLM application testing"', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 8, 'total_tokens': 16}, 'model_name': 'asiansoul_q8_0/Joah-Remix-Llama-3-KoEn-8B-Reborn-8B-Q8_0', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-94495e6d-8b88-4408-a375-39ed769d675b-0', usage_metadata={'input_tokens': 8, 'output_tokens': 8, 'total_tokens': 16})

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

#Runnablebranch -> if문 형식의 분기처리를 지원 if와 else는 따로 표기하지 않는듯?'
query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1,
        # If only one message, then we just pass that message's content to retriever, 대화가 하나이면 llm을 사용하지 않고 그냥 검색
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
    query_transform_prompt | chat | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

In [16]:
SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context. 
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

'''
query_transforming_retriever_chain:
query_transform_prompt | chat | StrOutputParser() | retriever
그간의 메시지를 이용, 검색의 위한 쿼리를 생성하고 검색을 실행,
이후 document chain에 해당 내용을 전달, 답변을 생성합니다.
'''
conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

In [17]:
response = conversational_retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Can LangSmith help test my LLM applications?"),
        ]
    }
)
response['answer']


'Yes, LangSmith can help you test your LLM (Large Language Model) applications. It provides a platform for building production-grade LLM applications and allows you to closely monitor and evaluate them so that you can ship quickly and with confidence. LangSmith also offers multiple ways to log traces to the platform, which enables you to track the performance of your application and identify any issues or areas for improvement. Additionally, LangSmith provides how-to guides on evaluation, which can help you understand how to use the platform effectively for testing your LLM applications. Overall, LangSmith is a useful tool for developers who want to build and test high-quality LLM applications.'

In [24]:
conversational_retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Can LangSmith help test my LLM applications?"),
            AIMessage(
                content=response['answer']
            ),
            HumanMessage(content="Tell me more!"),
        ],
    }
)['answer']

"LangSmith offers several features that make it an excellent choice for testing LLM applications:\n\n1. **Evaluation**: LangSmith provides a built-in evaluation system that allows you to assess the performance of your LLM application. You can use this feature to evaluate your model's accuracy, fluency, and coherence.\n\n2. **Tracing**: LangSmith offers multiple ways to log traces to the platform, which enables you to track the performance of your application and identify any issues or areas for improvement. This feature is particularly useful when debugging complex LLM applications.\n\n3. **Monitoring**: LangSmith provides real-time monitoring capabilities that allow you to track the status of your application and receive alerts if there are any issues.\n\n4. **API Key Management**: LangSmith allows you to manage API keys securely, which is essential for protecting sensitive data and ensuring the integrity of your LLM applications.\n\n5. **Community Support**: LangSmith has an active c